In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import gensim
from gensim import matutils , models , corpora
import scipy.sparse

C:\Users\Abhay Mahajan\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [95]:
df_maps = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\ParkReviewsLang.csv")

In [96]:
df_subway = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\SubwayReviewsLang.csv")

In [97]:
df_yelp = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\YelpParkReviewsScraped.csv")

In [98]:
df_maps.head()

,Unnamed: 0,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text,lang
0,0,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUNpeGF6TTNnRRAB,Claudia,https://www.google.com/maps/contrib/1001449741...,7 months ago,2021-06-20 22:04:09.211296,4.0,107.0,One of the nicest entry points to this invitin...,en
1,1,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSURDOGEyMGpnRRAB,Nate Neel,https://www.google.com/maps/contrib/1121030547...,8 months ago,2021-06-20 22:04:09.212245,5.0,121.0,"Waterfront to fish or just relax, great place ...",en
2,2,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUM4Nk9Ya3lnRRAB,Yucel Salimoglu,https://www.google.com/maps/contrib/1034180738...,11 months ago,2021-06-20 22:04:09.213178,4.0,79.0,Everything except the parking is good here.,en
3,3,Parc de la Capture-d'Ethan-Allen,ChZDSUhNMG9nS0VJQ0FnSUNVdWNUbE9REAE,COCO BEADZ,https://www.google.com/maps/contrib/1036060504...,a year ago,2021-06-20 22:04:09.214115,4.0,128.0,"Defenely the best park in Montreal East, Tetre...",en
4,4,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUMwdHJDTm1nRRAB,Anna Maria Fiore,https://www.google.com/maps/contrib/1016779009...,a year ago,2021-06-20 22:04:09.215069,5.0,39.0,It's so peaceful and happy place near the water,en


In [99]:
df_subway = df_subway[df_subway['lang'] == 'en']
df_subway.head()

,Unnamed: 0,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text,lang
0,0,Lionel-Groulx,ChZDSUhNMG9nS0VJQ0FnSUNjdjkzOVZREAE,Jisan Samrat,https://www.google.com/maps/contrib/1182926626...,a year ago,2021-06-30 00:37:13.278544,5.0,109,Green Line and Orange Line,en
1,1,Lionel-Groulx,ChdDSUhNMG9nS0VJQ0FnSUR5enRHYW13RRAB,Sheila Ferrando,https://www.google.com/maps/contrib/1096381594...,3 months ago,2021-06-30 00:37:13.279844,5.0,82,It's w metro station Google!,en
2,2,Lionel-Groulx,ChZDSUhNMG9nS0VJQ0FnSUNxeHVpR0NnEAE,John Mandrake,https://www.google.com/maps/contrib/1049594479...,3 weeks ago,2021-06-30 00:37:13.283496,4.0,47,Good metro station man,en
3,3,Lionel-Groulx,ChdDSUhNMG9nS0VJQ0FnSURLdXByYmdBRRAB,Abdoulie Touray,https://www.google.com/maps/contrib/1108310335...,a month ago,2021-06-30 00:37:13.285167,5.0,6,Easily accessible,en
4,4,Lionel-Groulx,ChRDSUhNMG9nS0VJQ0FnSURLdExZUBAB,George Swanson,https://www.google.com/maps/contrib/1165742509...,a month ago,2021-06-30 00:37:13.286322,5.0,8,Nice metro,en


In [100]:
df_yelp.head()

,Unnamed: 0,Comment,Rating,Rating Date,Profile Name,Profile Link,Profile Location,Place
0,0,This is a winter review. For all winter activ...,5,1/16/2021,Aimee H.,https://www.yelp.com/user_details?userid=oW9Po...,"Montreal, Canada",Parc du Mont-Royal
1,1,Parc du Royal has a character all on its own a...,5,8/23/2020,Mercedes C.,https://www.yelp.com/user_details?userid=ooHF-...,"Bronx, NY",Parc du Mont-Royal
2,2,This park provides a fun amount of physical ac...,5,8/28/2020,John S.,https://www.yelp.com/user_details?userid=xHMq2...,"Toronto, Canada",Parc du Mont-Royal
3,3,Parc du Mont-Royal or Mount Royal Park is like...,4,2/8/2020,Daniel B.,https://www.yelp.com/user_details?userid=j14Wg...,"Atlanta, GA",Parc du Mont-Royal
4,4,Really nice urban park with amazing views of t...,5,9/17/2020,Amy L.,https://www.yelp.com/user_details?userid=TfLxu...,"Toronto, Canada",Parc du Mont-Royal


In [101]:
df_yelp = df_yelp['Comment']
df_subway = df_subway[df_subway['lang'] == 'en']
df_subway = df_subway['review_text']

In [102]:
df_all = df_maps[df_maps['lang'] == 'en']
df_all = df_all['review_text']
df_all.append(df_yelp)
df_all.append(df_subway)
df_all = pd.DataFrame(df_all)

In [103]:
df_all.head()

,review_text
0,One of the nicest entry points to this invitin...
1,"Waterfront to fish or just relax, great place ..."
2,Everything except the parking is good here.
3,"Defenely the best park in Montreal East, Tetre..."
4,It's so peaceful and happy place near the water


In [105]:
df_all.to_csv('topic-modeling.csv')

In [71]:
df_all.shape

(23032, 1)

### Topic Modeling - Attempt #1 (Nouns and Adjectives) - Entire Dataset

In [74]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to C:\Users\Abhay
[nltk_data]     Mahajan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Abhay Mahajan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [75]:
# Let's create a function to pull out nouns and adjectives from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [76]:
df_all = pd.DataFrame(df_all['review_text'].apply(nouns_adj))

In [77]:
ps = nltk.stem.PorterStemmer()

### Stemming

In [78]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [79]:
df_all['review_text'] = df_all['review_text'].apply(stem_sentences)

In [80]:
df_all.head()

,review_text
0,nicest entri point waterfront promenad st-lawr...
1,waterfront great place hangout sunset amaz place
2,everyth park good
3,best park montreal east tetreauvil area park b...
4,peac happi place water


In [82]:
from sklearn.feature_extraction import text
add_stop_words = ['translated' , 'googl']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [83]:
from sklearn.feature_extraction.text import CountVectorizer
cv_noun_adj = CountVectorizer(stop_words = stop_words)
data_cv_noun_adj = cv_noun_adj.fit_transform(df_all['review_text']).toarray()
features_noun_adj = cv_noun_adj.get_feature_names()

In [84]:
df1 = pd.DataFrame(data_cv_noun_adj , columns = features_noun_adj)

In [85]:
term_document_matrix_noun_adj = df1.transpose()
term_document_matrix_noun_adj
sparse_matrix_noun_adj = scipy.sparse.csr_matrix(term_document_matrix_noun_adj)
corpus_noun_adj = matutils.Sparse2Corpus(sparse_matrix_noun_adj)

In [86]:
id2word_noun_adj = corpora.Dictionary([features_noun_adj])

In [91]:
# Number of Topics = 2
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=2, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.111*"park" + 0.030*"nice" + 0.025*"kid" + 0.022*"lot" + 0.016*"great" + 0.014*"montreal" + 0.014*"area" + 0.012*"water" + 0.012*"dog" + 0.011*"big"'),
 (1,
  '0.095*"place" + 0.047*"nice" + 0.046*"great" + 0.037*"beauti" + 0.034*"good" + 0.018*"famili" + 0.018*"walk" + 0.018*"parc" + 0.017*"origin" + 0.017*"time"')]

In [92]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-3.0915207901167725

In [93]:
# Number of Topics = 3
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=3, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.068*"beauti" + 0.048*"park" + 0.027*"montreal" + 0.021*"citi" + 0.019*"view" + 0.015*"best" + 0.012*"bike" + 0.010*"parc" + 0.010*"natur" + 0.010*"amaz"'),
 (1,
  '0.091*"park" + 0.035*"kid" + 0.034*"good" + 0.031*"lot" + 0.017*"dog" + 0.016*"area" + 0.016*"big" + 0.015*"water" + 0.015*"space" + 0.015*"great"'),
 (2,
  '0.148*"place" + 0.140*"nice" + 0.082*"great" + 0.045*"park" + 0.027*"origin" + 0.024*"famili" + 0.018*"walk" + 0.018*"friend" + 0.017*"super" + 0.017*"picnic"')]

In [94]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-3.1727085620245608

In [89]:
# Number of Topics = 10
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=10, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.270*"nice" + 0.218*"place" + 0.046*"quiet" + 0.039*"peac" + 0.031*"walk" + 0.029*"clean" + 0.021*"relax" + 0.014*"la" + 0.013*"ok" + 0.012*"air"'),
 (1,
  '0.028*"pond" + 0.026*"access" + 0.026*"duck" + 0.020*"metro" + 0.016*"st" + 0.014*"water" + 0.014*"swim" + 0.014*"easi" + 0.014*"festiv" + 0.014*"station"'),
 (2,
  '0.144*"great" + 0.112*"place" + 0.100*"beauti" + 0.059*"origin" + 0.036*"excel" + 0.032*"view" + 0.028*"walk" + 0.023*"natur" + 0.020*"amaz" + 0.019*"bike"'),
 (3,
  '0.061*"space" + 0.059*"park" + 0.049*"field" + 0.048*"lot" + 0.038*"soccer" + 0.036*"court" + 0.033*"sport" + 0.031*"picnic" + 0.030*"tenni" + 0.030*"green"'),
 (4,
  '0.089*"good" + 0.081*"park" + 0.045*"citi" + 0.040*"montreal" + 0.039*"best" + 0.028*"peopl" + 0.027*"beauti" + 0.027*"cool" + 0.021*"mani" + 0.016*"time"'),
 (5,
  '0.039*"park" + 0.027*"montreal" + 0.025*"trail" + 0.025*"littl" + 0.017*"walk" + 0.013*"cute" + 0.013*"island" + 0.013*"view" + 0.013*"citi" + 0.012*"small"'),
 (6,
  

In [90]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-5.425093178411784

### Topic Modeling - Attempt #1 (Nouns and Adjectives) - Negative Reviews

In [106]:
df_maps = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\ParkReviewsLang.csv")
df_subway = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\SubwayReviewsLang.csv")
df_yelp = pd.read_csv(r"C:\Users\Abhay Mahajan\Downloads\YelpParkReviewsScraped.csv")

In [107]:
df_maps.head()

,Unnamed: 0,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text,lang
0,0,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUNpeGF6TTNnRRAB,Claudia,https://www.google.com/maps/contrib/1001449741...,7 months ago,2021-06-20 22:04:09.211296,4.0,107.0,One of the nicest entry points to this invitin...,en
1,1,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSURDOGEyMGpnRRAB,Nate Neel,https://www.google.com/maps/contrib/1121030547...,8 months ago,2021-06-20 22:04:09.212245,5.0,121.0,"Waterfront to fish or just relax, great place ...",en
2,2,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUM4Nk9Ya3lnRRAB,Yucel Salimoglu,https://www.google.com/maps/contrib/1034180738...,11 months ago,2021-06-20 22:04:09.213178,4.0,79.0,Everything except the parking is good here.,en
3,3,Parc de la Capture-d'Ethan-Allen,ChZDSUhNMG9nS0VJQ0FnSUNVdWNUbE9REAE,COCO BEADZ,https://www.google.com/maps/contrib/1036060504...,a year ago,2021-06-20 22:04:09.214115,4.0,128.0,"Defenely the best park in Montreal East, Tetre...",en
4,4,Parc de la Capture-d'Ethan-Allen,ChdDSUhNMG9nS0VJQ0FnSUMwdHJDTm1nRRAB,Anna Maria Fiore,https://www.google.com/maps/contrib/1016779009...,a year ago,2021-06-20 22:04:09.215069,5.0,39.0,It's so peaceful and happy place near the water,en


In [108]:
df_yelp.head()

,Unnamed: 0,Comment,Rating,Rating Date,Profile Name,Profile Link,Profile Location,Place
0,0,This is a winter review. For all winter activ...,5,1/16/2021,Aimee H.,https://www.yelp.com/user_details?userid=oW9Po...,"Montreal, Canada",Parc du Mont-Royal
1,1,Parc du Royal has a character all on its own a...,5,8/23/2020,Mercedes C.,https://www.yelp.com/user_details?userid=ooHF-...,"Bronx, NY",Parc du Mont-Royal
2,2,This park provides a fun amount of physical ac...,5,8/28/2020,John S.,https://www.yelp.com/user_details?userid=xHMq2...,"Toronto, Canada",Parc du Mont-Royal
3,3,Parc du Mont-Royal or Mount Royal Park is like...,4,2/8/2020,Daniel B.,https://www.yelp.com/user_details?userid=j14Wg...,"Atlanta, GA",Parc du Mont-Royal
4,4,Really nice urban park with amazing views of t...,5,9/17/2020,Amy L.,https://www.yelp.com/user_details?userid=TfLxu...,"Toronto, Canada",Parc du Mont-Royal


In [109]:
df_subway.head()

,Unnamed: 0,review_for,review_id,username,user_url,published,date_retrieved,num_stars,num_reviews,review_text,lang
0,0,Lionel-Groulx,ChZDSUhNMG9nS0VJQ0FnSUNjdjkzOVZREAE,Jisan Samrat,https://www.google.com/maps/contrib/1182926626...,a year ago,2021-06-30 00:37:13.278544,5.0,109,Green Line and Orange Line,en
1,1,Lionel-Groulx,ChdDSUhNMG9nS0VJQ0FnSUR5enRHYW13RRAB,Sheila Ferrando,https://www.google.com/maps/contrib/1096381594...,3 months ago,2021-06-30 00:37:13.279844,5.0,82,It's w metro station Google!,en
2,2,Lionel-Groulx,ChZDSUhNMG9nS0VJQ0FnSUNxeHVpR0NnEAE,John Mandrake,https://www.google.com/maps/contrib/1049594479...,3 weeks ago,2021-06-30 00:37:13.283496,4.0,47,Good metro station man,en
3,3,Lionel-Groulx,ChdDSUhNMG9nS0VJQ0FnSURLdXByYmdBRRAB,Abdoulie Touray,https://www.google.com/maps/contrib/1108310335...,a month ago,2021-06-30 00:37:13.285167,5.0,6,Easily accessible,en
4,4,Lionel-Groulx,ChRDSUhNMG9nS0VJQ0FnSURLdExZUBAB,George Swanson,https://www.google.com/maps/contrib/1165742509...,a month ago,2021-06-30 00:37:13.286322,5.0,8,Nice metro,en


In [111]:
df_maps = df_maps[df_maps['num_stars'] < 4]
df_yelp = df_yelp[df_yelp['Rating'] < 4]
df_subway = df_subway[df_subway['num_stars'] < 4]

In [112]:
df_yelp = df_yelp['Comment']
df_subway = df_subway[df_subway['lang'] == 'en']
df_subway = df_subway['review_text']

In [113]:
df_all = df_maps[df_maps['lang'] == 'en']
df_all = df_all['review_text']
df_all.append(df_yelp)
df_all.append(df_subway)
df_all = pd.DataFrame(df_all)

In [114]:
df_all.shape

(2248, 1)

In [115]:
df_all = pd.DataFrame(df_all['review_text'].apply(nouns_adj))

In [116]:
ps = nltk.stem.PorterStemmer()

### Stemming

In [117]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [118]:
df_all['review_text'] = df_all['review_text'].apply(stem_sentences)

In [119]:
from sklearn.feature_extraction import text
add_stop_words = ['translated' , 'googl']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

In [120]:
from sklearn.feature_extraction.text import CountVectorizer
cv_noun_adj = CountVectorizer(stop_words = stop_words)
data_cv_noun_adj = cv_noun_adj.fit_transform(df_all['review_text']).toarray()
features_noun_adj = cv_noun_adj.get_feature_names()

In [121]:
df2 = pd.DataFrame(data_cv_noun_adj , columns = features_noun_adj)

In [125]:
term_document_matrix_noun_adj = df2.transpose()
term_document_matrix_noun_adj
sparse_matrix_noun_adj = scipy.sparse.csr_matrix(term_document_matrix_noun_adj)
corpus_noun_adj = matutils.Sparse2Corpus(sparse_matrix_noun_adj)

In [126]:
id2word_noun_adj = corpora.Dictionary([features_noun_adj])

In [127]:
# Number of Topics = 2
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=2, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.034*"nice" + 0.028*"place" + 0.020*"park" + 0.013*"ok" + 0.013*"bad" + 0.013*"area" + 0.010*"dirti" + 0.008*"parc" + 0.008*"montreal" + 0.007*"small"'),
 (1,
  '0.090*"park" + 0.026*"kid" + 0.021*"nice" + 0.019*"peopl" + 0.017*"good" + 0.016*"great" + 0.015*"lot" + 0.014*"water" + 0.014*"place" + 0.012*"mani"')]

In [128]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-4.633133572901304

In [129]:
# Number of Topics = 3
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=3, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.029*"good" + 0.017*"mani" + 0.014*"park" + 0.014*"littl" + 0.014*"bad" + 0.012*"area" + 0.011*"noth" + 0.010*"origin" + 0.009*"spot" + 0.009*"bit"'),
 (1,
  '0.103*"park" + 0.046*"nice" + 0.032*"place" + 0.023*"kid" + 0.016*"peopl" + 0.015*"water" + 0.014*"dog" + 0.014*"lot" + 0.012*"small" + 0.009*"great"'),
 (2,
  '0.024*"ok" + 0.017*"great" + 0.016*"parc" + 0.011*"cool" + 0.011*"kid" + 0.009*"metro" + 0.008*"day" + 0.007*"origin" + 0.006*"ice" + 0.006*"super"')]

In [130]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-8.252994560326371

In [131]:
# Number of Topics = 10
lda_noun_adj = models.LdaModel(corpus=corpus_noun_adj, id2word=id2word_noun_adj, num_topics=10, passes=40)
lda_noun_adj.print_topics()

[(0,
  '0.024*"suit" + 0.019*"bath" + 0.017*"new" + 0.017*"mainten" + 0.016*"day" + 0.016*"pool" + 0.015*"mai" + 0.014*"kid" + 0.013*"onli" + 0.013*"swimsuit"'),
 (1,
  '0.087*"park" + 0.057*"lot" + 0.038*"small" + 0.032*"peopl" + 0.018*"homeless" + 0.015*"mani" + 0.014*"neighborhood" + 0.012*"free" + 0.011*"hous" + 0.010*"junki"'),
 (2,
  '0.043*"mani" + 0.042*"park" + 0.042*"peopl" + 0.024*"garbag" + 0.015*"food" + 0.013*"area" + 0.013*"horribl" + 0.012*"distanc" + 0.011*"great" + 0.010*"safe"'),
 (3,
  '0.057*"park" + 0.049*"dog" + 0.038*"great" + 0.036*"water" + 0.033*"dirti" + 0.018*"bit" + 0.014*"car" + 0.012*"picnic" + 0.012*"access" + 0.012*"super"'),
 (4,
  '0.124*"park" + 0.052*"nice" + 0.048*"kid" + 0.021*"ok" + 0.017*"playground" + 0.017*"parc" + 0.017*"field" + 0.014*"citi" + 0.014*"small" + 0.014*"basebal"'),
 (5,
  '0.131*"good" + 0.034*"correct" + 0.032*"parc" + 0.025*"work" + 0.021*"le" + 0.019*"shape" + 0.017*"walkway" + 0.012*"eau" + 0.011*"bad" + 0.011*"job"'),
 (6,

In [132]:
gensim.models.coherencemodel.CoherenceModel(corpus = corpus_noun_adj , model=lda_noun_adj, coherence='u_mass').get_coherence()

-9.898181033530548